In [30]:
import pandas as pd
import numpy as np


In [31]:
data = pd.read_csv("../DATASETS/titanic/train.csv")

In [32]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## As by seeing the data , i think i don't need some of the columns in the above data like 'Name', 'Ticket' ,'Cabin' ,'Fare', 'PassengerId' (as it is just the serial no. ),   for checking that the given person would have survived or not , and the data in these columns is not that much good so... i will drop these columns 

In [33]:
# so i willl drop these columns
data.drop(["Name","PassengerId","Ticket","Fare","Cabin"],axis=1,inplace=True)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [34]:
# to get information about data of all the columns   
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Embarked    889 non-null object
dtypes: float64(1), int64(4), object(2)
memory usage: 48.8+ KB


### so as seen above 'Age' column have around 170 values that are null so we need to fill those values for prediction as age can be an important factor 

In [35]:
# so lets first check the uniqu values in embarked column 
data["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [36]:
#now replacing the data (string) with numeric values as i need numeric values
data.replace("male",1,inplace=True)
data.replace("female",0,inplace=True)
data.replace("S",0,inplace=True)
data.replace("C",1,inplace=True)
data.replace("Q",2,inplace=True)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,1,22.0,1,0,0.0
1,1,1,0,38.0,1,0,1.0
2,1,3,0,26.0,0,0,0.0
3,1,1,0,35.0,1,0,0.0
4,0,3,1,35.0,0,0,0.0


In [37]:
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
X=data.iloc[:,1:]
y=data.iloc[:,0]

In [38]:
imp = Imputer(missing_values = "NaN", strategy = "most_frequent",axis=0)
imp.fit(X)
X=imp.transform(X)

# by knn

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
steps = [ ('imp', Imputer(missing_values="NaN",strategy="mean",axis=0)),
          ('clf', KNeighborsClassifier(n_neighbors = 6))]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

pipeline = Pipeline(steps)

pipeline.fit(X_train, y_train)
pipeline.predict(X_test)
pipeline.score(X_test, y_test)

0.79329608938547491

# by Decision Tree

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
steps = [ ('imp', Imputer(missing_values="NaN",strategy="mean",axis=0)),
          ('dt', DecisionTreeClassifier())]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, random_state=42)

pipeline = Pipeline(steps)

pipeline.fit(X_train, y_train)
pipeline.predict(X_test)
pipeline.score(X_test, y_test)

0.77871148459383754

# by Xgboost

In [44]:
import xgboost as xgb
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

xg_cl = xgb.XGBClassifier(objective="binary:logistic",n_estimators = 10,seed=123)

xg_cl.fit(X_train,y_train)

preds = xg_cl.predict(X_test)

accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy:  ", (accuracy))

accuracy:   0.8379888268156425


# by random forest

In [45]:
from sklearn.ensemble import RandomForestClassifier


clf=RandomForestClassifier(bootstrap=True,
            max_depth=6, max_features='auto', 
             n_estimators=10, random_state=42)
clf.fit(X_train,y_train)
clf.predict(X_test)
clf.score(X_test, y_test)



0.85474860335195535

# Validation

In [46]:
dc = pd.read_csv("../DATASETS/titanic/test.csv")
dx = pd.read_csv("../DATASETS/titanic/gender_submission.csv")
print(dx.head())
dc.head()

   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [47]:
dc.drop(["PassengerId","Name","Ticket","Fare","Cabin"],axis=1,inplace=True)
dc.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,male,34.5,0,0,Q
1,3,female,47.0,1,0,S
2,2,male,62.0,0,0,Q
3,3,male,27.0,0,0,S
4,3,female,22.0,1,1,S


In [48]:
dc.replace("male",1,inplace=True)
dc.replace("female",0,inplace=True)
dc.replace("S",0,inplace=True)
dc.replace("C",1,inplace=True)
dc.replace("Q",2,inplace=True)

In [49]:
dc.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,1,34.5,0,0,2
1,3,0,47.0,1,0,0
2,2,1,62.0,0,0,2
3,3,1,27.0,0,0,0
4,3,0,22.0,1,1,0


In [50]:
dc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass      418 non-null int64
Sex         418 non-null int64
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Embarked    418 non-null int64
dtypes: float64(1), int64(5)
memory usage: 19.7 KB


# KNN

In [51]:
from sklearn.preprocessing import Imputer
X_test=dc.iloc[:,:]
y_test=dx.iloc[:,1]
imp = Imputer(missing_values = "NaN", strategy = "most_frequent",axis=0)
imp.fit(X)
X=imp.transform(X)
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
steps = [ ('imp', Imputer(missing_values="NaN",strategy="mean",axis=0)),
          ('clf', KNeighborsClassifier(n_neighbors = 6))]
pipeline = Pipeline(steps)

pipeline.fit(X_train, y_train)
pipeline.predict(X_test)
pipeline.score(X_test,y_test)



0.76315789473684215

# Decision Tree

In [52]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
steps = [ ('imp', Imputer(missing_values="NaN",strategy="mean",axis=0)),
          ('dt', DecisionTreeClassifier())]
X_test=dc.iloc[:,:]
y_test=dx.iloc[:,1]

pipeline = Pipeline(steps)

pipeline.fit(X_train, y_train)
pipeline.predict(X_test)
pipeline.score(X_test, y_test)

0.73923444976076558

# XG boost

In [53]:
import xgboost as xgb
X_train = data.iloc[:,1:]
y_train = data.iloc[:,0]
X_test=dc.iloc[:,:]
y_test=dx.iloc[:,1]


xg_cl = xgb.XGBClassifier(objective="binary:logistic",n_estimators = 10,seed=123)

xg_cl.fit(X_train,y_train)

preds = xg_cl.predict(X_test)

accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy:  ", (accuracy))


accuracy:   0.8779904306220095
